## **Parametric Models and Regressors** 

Please stay tuned, here we will have the parametric version of the models...

In [1]:
from robot_descriptions.ur5_description import URDF_PATH

from darli.parametric import Functional
from darli.backend import CasadiBackend
from darli.robots import manipulator
from darli.functional import FunctionalStateSpace

In [2]:
manip = manipulator(Functional, CasadiBackend, URDF_PATH)


In [3]:
manip.inertia

Function(inertia:(q[6],parameters[60])->(inertia[6x6]) SXFunction)

In [4]:
manip.forward_dynamics

Function(forward_dynamics:(q[6],v[6],tau[6],parameters[60])->(dv[6]) SXFunction)

In [5]:
state_space = FunctionalStateSpace(manip)

In [6]:
state_space.derivative

Function(derivative:(q[6],v[6],tau[6],theta[60])->(derivative[12]) SXFunction)